<a href="https://colab.research.google.com/github/himanshudce/APM_BPI_2020/blob/main/notebooks/model_building_and_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
import numpy as np
import sklearn
import torch
import torch.nn as nn
from tqdm.auto import tqdm


In [3]:
import pickle
import numpy

In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import accuracy_score
np.random.seed(42)

# Event Log & Data Loading

In [4]:
# function to load the data from pickle
def load_data(load_path):
    with open(load_path, 'rb') as handle:
        data = pickle.load(handle)
    return data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
data_test = load_data("/content/drive/MyDrive/APM_BPI_2022/boolean_encode_test_trace_len_6.pickle")
data_train = load_data("/content/drive/MyDrive/APM_BPI_2022/boolean_encode_train_trace_len_6.pickle")

# Model Building

## Scaling
If needed, would take place here. Not required for binary values.

In [14]:
X_train = data_train['X']
X_test = data_test['X']
y_train = data_train['y']
y_test = data_test['y']

In [27]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

2825
2825
2825
460


In [12]:
# scaler_x = MinMaxScaler()
# data_scaled = scaler_x.fit_transform(data_dict['X'])

# scaler_y = FunctionTransformer() # for binary values scaling does not make sense at all but we keep it for symetry and apply the "NoOp" scaler
# target_scaled = scaler_y.fit_transform(data_dict['y'].reshape(-1, 1))

AttributeError: ignored

## Setting up Model Evaluation DataFrame
All the results will be saved to the `results_df` dataframe for comparison.

In [35]:
results_df = pd.DataFrame(index=['Decision Tree (Default)'],
                          columns=['F-score', 'Precision','Recall', 'Accuracy'])

## Decision Tree Classifier

In [38]:
# Create Decision Tree classifer object
dt = DecisionTreeClassifier()

# Train Decision Tree Classifer
dt_fit = dt.fit(X_train,y_train)

#Predict the response for test dataset
#dt_predict = dt_fit.predict(X_test)
dt_predict = dt_fit.predict(X_train)

#TODO: @Himanshu - using train data for evaluation because of incorrect test/train data setup.
#results_df.loc['Decision Tree (Default)',:] = [f1_score(y_test,dt_predict, average='macro'), precision_score(y_test,dt_predict), recall_score(y_test,dt_predict),accuracy_score(y_test,dt_predict)]
results_df.loc['Decision Tree (Default)',:] = [f1_score(y_train,dt_predict, average='macro'), precision_score(y_train,dt_predict), recall_score(y_train,dt_predict),accuracy_score(y_train,dt_predict)]

results_df.sort_values(by='F-score', ascending=False)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F-score,Precision,Recall,Accuracy
Decision Tree (Default),0.454001,0.0,0.0,0.831504


# Logistic Regression


In [39]:
# Training the Logistic Regression model on the Training set
lr = LogisticRegression(random_state=42)
lr_fit = lr.fit(X_train, y_train)
# Predicting the Test set results
#lr_predict = lr_fit.predict(X_test)
lr_predict = lr_fit.predict(X_train)

#results_df.loc['Logistic Regression (Default)',:] = [f1_score(y_test,lr_predict, average='macro'), precision_score(y_test,lr_predict), recall_score(y_test,lr_predict),accuracy_score(y_test,lr_predict)]
results_df.loc['Logistic Regression (Default)',:] = [f1_score(y_train,lr_predict, average='macro'), precision_score(y_train,lr_predict), recall_score(y_train,lr_predict),accuracy_score(y_train,lr_predict)]
results_df.sort_values(by='F-score', ascending=False)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F-score,Precision,Recall,Accuracy
Decision Tree (Default),0.454001,0.0,0.0,0.831504
Logistic Regression (Default),0.454001,0.0,0.0,0.831504


# Random Forest Classifier

In [42]:
rf = RandomForestClassifier(n_estimators=20, random_state=42, max_depth=4)
rf_fit = rf.fit(X_train, y_train)
# Predicting the Test set results
#rf_predict = rf_fit.predict(X_test)
rf_predict = rf_fit.predict(X_train)

#results_df.loc['Random Forest (Default)',:] = [f1_score(y_test,rf_predict, average='macro'), precision_score(y_test,rf_predict), recall_score(y_test,rf_predict),accuracy_score(y_test,rf_predict)]
results_df.loc['Random Forest (Default)',:] = [f1_score(y_train,rf_predict, average='macro'), precision_score(y_train,rf_predict), recall_score(y_train,rf_predict),accuracy_score(y_train,rf_predict)]
results_df.sort_values(by='F-score', ascending=False)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F-score,Precision,Recall,Accuracy
Decision Tree (Default),0.454001,0.0,0.0,0.831504
Logistic Regression (Default),0.454001,0.0,0.0,0.831504
Random Forest (Default),0.454001,0.0,0.0,0.831504


# MLP (Neural Netowork) Classifier

In [44]:
nnet = MLPClassifier(hidden_layer_sizes=[1],
                           alpha=0,
                           activation='logistic',
                           max_iter=400,
                           solver='lbfgs',random_state=42)

nnet_fit = nnet.fit(X_train, y_train)
#nnet_predict = nnet_fit.predict_proba(X_test)
#TODO: Decide if we want to predict outright, or predict probabilities?
#nnet_predict = nnet_fit.predict_proba(X_test)
nnet_predict = nnet_fit.predict(X_train)

results_df.loc['MLP (Default)',:] = [f1_score(y_train,nnet_predict, average='macro'), precision_score(y_train,nnet_predict), recall_score(y_train,nnet_predict),accuracy_score(y_train,nnet_predict)]
results_df.sort_values(by='F-score', ascending=False)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F-score,Precision,Recall,Accuracy
Decision Tree (Default),0.454001,0.0,0.0,0.831504
Logistic Regression (Default),0.454001,0.0,0.0,0.831504
Random Forest (Default),0.454001,0.0,0.0,0.831504
MLP (Default),0.454001,0.0,0.0,0.831504
